**References:**  

https://www.datacamp.com/community/tutorials/finance-python-trading

https://github.com/datacamp/datacamp-community-tutorials/blob/master/Python%20Finance%20Tutorial%20For%20Beginners/Python%20For%20Finance%20Beginners%20Tutorial.ipynb

https://pypi.python.org/pypi/fix-yahoo-finance

http://www.learndatasci.com/python-finance-part-yahoo-finance-api-pandas-matplotlib/

In [1]:
# Import initial libraries

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Imports in order to be able to use Plotly offline.
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

print(__version__) # requires version >= 1.9.0

init_notebook_mode(connected=True)

4.5.0


In [3]:
# Import the Sample worksheet with acquisition dates and initial cost basis:

portfolio_df = pd.read_excel('Sample stocks acquisition dates_costs.xlsx', sheet_name='Sample')

portfolio_df.head(10)

,Acquisition Date,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year
0,2013-02-07,AAPL,125,65.40,8175.00,2017-12-29
1,2014-02-27,JNJ,100,81.90,8190.00,2017-12-29
2,2013-06-04,MCD,100,84.99,8499.00,2017-12-29
3,2015-12-14,MTCH,600,13.63,8178.00,2017-12-29
4,2016-01-14,NFLX,75,108.71,8153.25,2017-12-29
5,2013-08-14,WMT,125,68.30,8537.50,2017-12-29
6,2013-12-13,FB,150,53.32,7998.00,2017-12-29
7,2015-01-05,TWTR,225,36.38,8185.50,2017-12-29


In [4]:
# Confirm that you have 8 values for each column.

portfolio_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Acquisition Date  8 non-null      datetime64[ns]
 1   Ticker            8 non-null      object        
 2   Quantity          8 non-null      int64         
 3   Unit Cost         8 non-null      float64       
 4   Cost Basis        8 non-null      float64       
 5   Start of Year     8 non-null      datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(1)
memory usage: 512.0+ bytes


In [5]:
# Date Ranges for SP 500 and for all tickers
# Modify these date ranges each week.

# The below will pull back stock prices from 2010 until end date specified.
start_sp = datetime.datetime(2013, 1, 1)
end_sp = datetime.datetime(2018, 3, 9)

# This variable is used for YTD performance.
end_of_last_year = datetime.datetime(2017, 12, 29)

# These are separate if for some reason want different date range than SP.
stocks_start = datetime.datetime(2013, 1, 1)
stocks_end = datetime.datetime(2018, 3, 9)

In [6]:
# Leveraged from the helpful Datacamp Python Finance trading blog post.

from pandas_datareader import data as pdr
#import fix_yahoo_finance as yf
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

sp500 = pdr.get_data_yahoo('^GSPC', 
                           start_sp,
                             end_sp)
                          
sp500.head()

/home/gary/git/finance/.env/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,1462.420044,4202600000
2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,1459.369995,3829730000
2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,1466.469971,3424290000
2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,1461.890015,3304970000
2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,1457.150024,3601600000


In [7]:
sp500.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-03-05,2681.060059,2728.090088,2675.750000,2720.939941,2720.939941,3710810000
2018-03-06,2730.179932,2732.080078,2711.260010,2728.120117,2728.120117,3370690000
2018-03-07,2710.179932,2730.600098,2701.739990,2726.800049,2726.800049,3393270000
2018-03-08,2732.750000,2740.449951,2722.649902,2738.969971,2738.969971,3212320000
2018-03-09,2752.909912,2786.570068,2751.540039,2786.570068,2786.570068,3364100000


In [8]:
# Create a dataframe with only the Adj Close column as that's all we need for this analysis.

sp_500_adj_close = sp500[['Adj Close']].reset_index()

In [9]:
sp_500_adj_close.tail()

,Date,Adj Close
1301,2018-03-05,2720.939941
1302,2018-03-06,2728.120117
1303,2018-03-07,2726.800049
1304,2018-03-08,2738.969971
1305,2018-03-09,2786.570068


In [10]:
# Adj Close for the EOY in 2017 in order to run comparisons versus stocks YTD performances.

sp_500_adj_close_start = sp_500_adj_close[sp_500_adj_close['Date']==end_of_last_year]
sp_500_adj_close_start

,Date,Adj Close
1258,2017-12-29,2673.610107


In [11]:
# Generate a dynamic list of tickers to pull from Yahoo Finance API based on the imported file with tickers.
tickers = portfolio_df['Ticker'].unique()
tickers

array(['AAPL', 'JNJ', 'MCD', 'MTCH', 'NFLX', 'WMT', 'FB', 'TWTR'],
      dtype=object)

In [12]:
# Stock comparison code

def get(tickers, startdate, enddate):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))
               
all_data = get(tickers, stocks_start, stocks_end)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [15]:
all_data.tail()

Open       High        Low      Close  Adj Close  \
Ticker Date                                                                
TWTR   2018-03-05  32.900002  34.950001  32.580002  34.580002  34.580002   
       2018-03-06  34.580002  34.950001  33.860001  34.430000  34.430000   
       2018-03-07  33.990002  35.820000  33.990002  35.759998  35.759998   
       2018-03-08  35.419998  35.840000  34.680000  34.849998  34.849998   
       2018-03-09  35.200001  35.500000  34.939999  35.349998  35.349998   

                     Volume  
Ticker Date                  
TWTR   2018-03-05  34840000  
       2018-03-06  22641000  
       2018-03-07  33656100  
       2018-03-08  24845300  
       2018-03-09  19119500

In [25]:
# Also only pulling the ticker, date and adj. close columns for our tickers.

adj_close = all_data[['Adj Close']].reset_index()
adj_close.head()

,Ticker,Date,Adj Close
0,AAPL,2013-01-02,68.850548
1,AAPL,2013-01-03,67.981491
2,AAPL,2013-01-04,66.087891
3,AAPL,2013-01-07,65.699158
4,AAPL,2013-01-08,65.875954


In [26]:
# Grabbing the ticker close from the end of last year
adj_close_start = adj_close[adj_close['Date']==end_of_last_year]
adj_close_start.head()

,Ticker,Date,Adj Close
1258,AAPL,2017-12-29,164.258896
2564,JNJ,2017-12-29,132.300919
3870,MCD,2017-12-29,163.912796
4449,MTCH,2017-12-29,29.799627
5755,NFLX,2017-12-29,191.960007


In [27]:
# Grab the latest stock close price

adj_close_latest = adj_close[adj_close['Date']==stocks_end]
adj_close_latest

,Ticker,Date,Adj Close
1305,AAPL,2018-03-09,175.405365
2611,JNJ,2018-03-09,127.506561
3917,MCD,2018-03-09,150.689667
4496,MTCH,2018-03-09,42.629364
5802,NFLX,2018-03-09,331.440002
7108,WMT,2018-03-09,85.449150
8414,FB,2018-03-09,185.229996
9505,TWTR,2018-03-09,35.349998


In [28]:
adj_close_latest.set_index('Ticker', inplace=True)
adj_close_latest.head()

,Date,Adj Close
Ticker,,
AAPL,2018-03-09,175.405365
JNJ,2018-03-09,127.506561
MCD,2018-03-09,150.689667
MTCH,2018-03-09,42.629364
NFLX,2018-03-09,331.440002


In [29]:
portfolio_df.set_index(['Ticker'], inplace=True)

portfolio_df.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
Ticker,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2017-12-29
JNJ,2014-02-27,100,81.90,8190.00,2017-12-29
MCD,2013-06-04,100,84.99,8499.00,2017-12-29
MTCH,2015-12-14,600,13.63,8178.00,2017-12-29
NFLX,2016-01-14,75,108.71,8153.25,2017-12-29


In [30]:
# Merge the portfolio dataframe with the adj close dataframe; they are being joined by their indexes.

merged_portfolio = pd.merge(portfolio_df, adj_close_latest, left_index=True, right_index=True)
merged_portfolio.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close
Ticker,,,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365
JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561
MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667
MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364
NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002


In [31]:
# The below creates a new column which is the ticker return; takes the latest adjusted close for each position
# and divides that by the initial share cost.

merged_portfolio['ticker return'] = merged_portfolio['Adj Close'] / merged_portfolio['Unit Cost'] - 1

merged_portfolio

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close,ticker return
Ticker,,,,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365,1.682039
JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561,0.556857
MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667,0.773028
MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613
NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846
WMT,2013-08-14,125,68.30,8537.50,2017-12-29,2018-03-09,85.449150,0.251086
FB,2013-12-13,150,53.32,7998.00,2017-12-29,2018-03-09,185.229996,2.473931
TWTR,2015-01-05,225,36.38,8185.50,2017-12-29,2018-03-09,35.349998,-0.028312


In [32]:
merged_portfolio.reset_index(inplace=True)

In [33]:
# Here we are merging the new dataframe with the sp500 adjusted closes since the sp start price based on 
# each ticker's acquisition date and sp500 close date.

merged_portfolio_sp = pd.merge(merged_portfolio, sp_500_adj_close, left_on='Acquisition Date', right_on='Date')
# .set_index('Ticker')

merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date_x,Adj Close_x,ticker return,Date_y,Adj Close_y
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365,1.682039,2013-02-07,1509.390015
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561,0.556857,2014-02-27,1854.290039
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667,0.773028,2013-06-04,1631.380005
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2015-12-14,2021.939941
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,2016-01-14,1921.839966


In [34]:
# We will delete the additional date column which is created from this merge.
# We then rename columns to Latest Date and then reflect Ticker Adj Close and SP 500 Initial Close.

del merged_portfolio_sp['Date_y']

merged_portfolio_sp.rename(columns={'Date_x': 'Latest Date', 'Adj Close_x': 'Ticker Adj Close'
                                    , 'Adj Close_y': 'SP 500 Initial Close'}, inplace=True)

merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365,1.682039,1509.390015
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561,0.556857,1854.290039
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667,0.773028,1631.380005
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966


In [35]:
# This new column determines what SP 500 equivalent purchase would have been at purchase date of stock.
merged_portfolio_sp['Equiv SP Shares'] = merged_portfolio_sp['Cost Basis'] / merged_portfolio_sp['SP 500 Initial Close']
merged_portfolio_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365,1.682039,1509.390015,5.416095
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561,0.556857,1854.290039,4.416785
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667,0.773028,1631.380005,5.209700
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,4.044631
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,4.242419


In [36]:
# We are joining the developing dataframe with the sp500 closes again, this time with the latest close for SP.
merged_portfolio_sp_latest = pd.merge(merged_portfolio_sp, sp_500_adj_close, left_on='Latest Date', right_on='Date')

merged_portfolio_sp_latest.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,Date,Adj Close
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365,1.682039,1509.390015,5.416095,2018-03-09,2786.570068
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561,0.556857,1854.290039,4.416785,2018-03-09,2786.570068
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667,0.773028,1631.380005,5.209700,2018-03-09,2786.570068
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,4.044631,2018-03-09,2786.570068
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,4.242419,2018-03-09,2786.570068


In [37]:
# Once again need to delete the new Date column added as it's redundant to Latest Date.  
# Modify Adj Close from the sp dataframe to distinguish it by calling it the SP 500 Latest Close.

del merged_portfolio_sp_latest['Date']

merged_portfolio_sp_latest.rename(columns={'Adj Close': 'SP 500 Latest Close'}, inplace=True)

merged_portfolio_sp_latest.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365,1.682039,1509.390015,5.416095,2786.570068
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561,0.556857,1854.290039,4.416785,2786.570068
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667,0.773028,1631.380005,5.209700,2786.570068
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,4.044631,2786.570068
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,4.242419,2786.570068


In [38]:
# Percent return of SP from acquisition date of position through latest trading day.
merged_portfolio_sp_latest['SP Return'] = merged_portfolio_sp_latest['SP 500 Latest Close'] / merged_portfolio_sp_latest['SP 500 Initial Close'] - 1

# This is a new column which takes the tickers return and subtracts the sp 500 equivalent range return.
merged_portfolio_sp_latest['Abs. Return Compare'] = merged_portfolio_sp_latest['ticker return'] - merged_portfolio_sp_latest['SP Return']

# This is a new column where we calculate the ticker's share value by multiplying the original quantity by the latest close.
merged_portfolio_sp_latest['Ticker Share Value'] = merged_portfolio_sp_latest['Quantity'] * merged_portfolio_sp_latest['Ticker Adj Close']

# We calculate the equivalent SP 500 Value if we take the original SP shares * the latest SP 500 share price.
merged_portfolio_sp_latest['SP 500 Value'] = merged_portfolio_sp_latest['Equiv SP Shares'] * merged_portfolio_sp_latest['SP 500 Latest Close']

# This is a new column where we take the current market value for the shares and subtract the SP 500 value.
merged_portfolio_sp_latest['Abs Value Compare'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['SP 500 Value']

# This column calculates profit / loss for stock position.
merged_portfolio_sp_latest['Stock Gain / (Loss)'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['Cost Basis']

# This column calculates profit / loss for SP 500.
merged_portfolio_sp_latest['SP 500 Gain / (Loss)'] = merged_portfolio_sp_latest['SP 500 Value'] - merged_portfolio_sp_latest['Cost Basis']

merged_portfolio_sp_latest.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss)
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365,1.682039,1509.390015,5.416095,2786.570068,0.846156,0.835883,21925.670624,15092.328747,6833.341877,13750.670624,6917.328747
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561,0.556857,1854.290039,4.416785,2786.570068,0.502769,0.054087,12750.656128,12307.680233,442.975895,4560.656128,4117.680233
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667,0.773028,1631.380005,5.209700,2786.570068,0.708106,0.064922,15068.966675,14517.193382,551.773292,6569.966675,6018.193382
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,4.044631,2786.570068,0.378167,1.749446,25577.618408,11270.646349,14306.972059,17399.618408,3092.646349
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,4.242419,2786.570068,0.449949,1.598897,24858.000183,11821.797243,13036.202940,16704.750183,3668.547243


In [39]:
# Merge the overall dataframe with the adj close start of year dataframe for YTD tracking of tickers.
# Should not need to do the outer join;

merged_portfolio_sp_latest_YTD = pd.merge(merged_portfolio_sp_latest, adj_close_start, on='Ticker')
# , how='outer'

merged_portfolio_sp_latest_YTD.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Date,Adj Close
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365,1.682039,1509.390015,...,2786.570068,0.846156,0.835883,21925.670624,15092.328747,6833.341877,13750.670624,6917.328747,2017-12-29,164.258896
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561,0.556857,1854.290039,...,2786.570068,0.502769,0.054087,12750.656128,12307.680233,442.975895,4560.656128,4117.680233,2017-12-29,132.300919
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667,0.773028,1631.380005,...,2786.570068,0.708106,0.064922,15068.966675,14517.193382,551.773292,6569.966675,6018.193382,2017-12-29,163.912796
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,...,2786.570068,0.378167,1.749446,25577.618408,11270.646349,14306.972059,17399.618408,3092.646349,2017-12-29,29.799627
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,...,2786.570068,0.449949,1.598897,24858.000183,11821.797243,13036.202940,16704.750183,3668.547243,2017-12-29,191.960007


In [40]:
# Deleting date again as it's an unnecessary column.  Explaining that new column is the Ticker Start of Year Close.

del merged_portfolio_sp_latest_YTD['Date']

merged_portfolio_sp_latest_YTD.rename(columns={'Adj Close': 'Ticker Start Year Close'}, inplace=True)

merged_portfolio_sp_latest_YTD.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365,1.682039,1509.390015,5.416095,2786.570068,0.846156,0.835883,21925.670624,15092.328747,6833.341877,13750.670624,6917.328747,164.258896
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561,0.556857,1854.290039,4.416785,2786.570068,0.502769,0.054087,12750.656128,12307.680233,442.975895,4560.656128,4117.680233,132.300919
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667,0.773028,1631.380005,5.209700,2786.570068,0.708106,0.064922,15068.966675,14517.193382,551.773292,6569.966675,6018.193382,163.912796
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,4.044631,2786.570068,0.378167,1.749446,25577.618408,11270.646349,14306.972059,17399.618408,3092.646349,29.799627
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,4.242419,2786.570068,0.449949,1.598897,24858.000183,11821.797243,13036.202940,16704.750183,3668.547243,191.960007


In [41]:
# Join the SP 500 start of year with current dataframe for SP 500 ytd comparisons to tickers.

merged_portfolio_sp_latest_YTD_sp = pd.merge(merged_portfolio_sp_latest_YTD, sp_500_adj_close_start
                                             , left_on='Start of Year', right_on='Date')

merged_portfolio_sp_latest_YTD_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,Date,Adj Close
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365,1.682039,1509.390015,...,0.846156,0.835883,21925.670624,15092.328747,6833.341877,13750.670624,6917.328747,164.258896,2017-12-29,2673.610107
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561,0.556857,1854.290039,...,0.502769,0.054087,12750.656128,12307.680233,442.975895,4560.656128,4117.680233,132.300919,2017-12-29,2673.610107
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667,0.773028,1631.380005,...,0.708106,0.064922,15068.966675,14517.193382,551.773292,6569.966675,6018.193382,163.912796,2017-12-29,2673.610107
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,...,0.378167,1.749446,25577.618408,11270.646349,14306.972059,17399.618408,3092.646349,29.799627,2017-12-29,2673.610107
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,...,0.449949,1.598897,24858.000183,11821.797243,13036.202940,16704.750183,3668.547243,191.960007,2017-12-29,2673.610107


In [42]:
# Deleting another unneeded Date column.

del merged_portfolio_sp_latest_YTD_sp['Date']

# Renaming so that it's clear this column is SP 500 start of year close.
merged_portfolio_sp_latest_YTD_sp.rename(columns={'Adj Close': 'SP Start Year Close'}, inplace=True)

# YTD return for portfolio position.
merged_portfolio_sp_latest_YTD_sp['Share YTD'] = merged_portfolio_sp_latest_YTD_sp['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp['Ticker Start Year Close'] - 1

# YTD return for SP to run compares.
merged_portfolio_sp_latest_YTD_sp['SP 500 YTD'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Latest Close'] / merged_portfolio_sp_latest_YTD_sp['SP Start Year Close'] - 1

merged_portfolio_sp_latest_YTD_sp.head()

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365,1.682039,1509.390015,...,0.835883,21925.670624,15092.328747,6833.341877,13750.670624,6917.328747,164.258896,2673.610107,0.067859,0.04225
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561,0.556857,1854.290039,...,0.054087,12750.656128,12307.680233,442.975895,4560.656128,4117.680233,132.300919,2673.610107,-0.036238,0.04225
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667,0.773028,1631.380005,...,0.064922,15068.966675,14517.193382,551.773292,6569.966675,6018.193382,163.912796,2673.610107,-0.080672,0.04225
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,...,1.749446,25577.618408,11270.646349,14306.972059,17399.618408,3092.646349,29.799627,2673.610107,0.430533,0.04225
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,...,1.598897,24858.000183,11821.797243,13036.202940,16704.750183,3668.547243,191.960007,2673.610107,0.726610,0.04225


In [43]:
merged_portfolio_sp_latest_YTD_sp = merged_portfolio_sp_latest_YTD_sp.sort_values(by='Ticker', ascending=True)
merged_portfolio_sp_latest_YTD_sp

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365,1.682039,1509.390015,...,0.835883,21925.670624,15092.328747,6833.341877,13750.670624,6917.328747,164.258896,2673.610107,0.067859,0.04225
6,FB,2013-12-13,150,53.32,7998.00,2017-12-29,2018-03-09,185.229996,2.473931,1775.319946,...,1.904315,27784.499359,12553.786405,15230.712954,19786.499359,4555.786405,176.460007,2673.610107,0.049700,0.04225
1,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561,0.556857,1854.290039,...,0.054087,12750.656128,12307.680233,442.975895,4560.656128,4117.680233,132.300919,2673.610107,-0.036238,0.04225
2,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667,0.773028,1631.380005,...,0.064922,15068.966675,14517.193382,551.773292,6569.966675,6018.193382,163.912796,2673.610107,-0.080672,0.04225
3,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,...,1.749446,25577.618408,11270.646349,14306.972059,17399.618408,3092.646349,29.799627,2673.610107,0.430533,0.04225
4,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,...,1.598897,24858.000183,11821.797243,13036.202940,16704.750183,3668.547243,191.960007,2673.610107,0.726610,0.04225
7,TWTR,2015-01-05,225,36.38,8185.50,2017-12-29,2018-03-09,35.349998,-0.028312,2020.579956,...,-0.407406,7953.749657,11288.575454,-3334.825797,-231.750343,3103.075454,24.010000,2673.610107,0.472303,0.04225
5,WMT,2013-08-14,125,68.30,8537.50,2017-12-29,2018-03-09,85.449150,0.251086,1685.390015,...,-0.402282,10681.143761,14115.630063,-3434.486302,2143.643761,5578.130063,94.545700,2673.610107,-0.096213,0.04225


In [44]:
# Cumulative sum of original investment
merged_portfolio_sp_latest_YTD_sp['Cum Invst'] = merged_portfolio_sp_latest_YTD_sp['Cost Basis'].cumsum()

# Cumulative sum of Ticker Share Value (latest FMV based on initial quantity purchased).
merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] = merged_portfolio_sp_latest_YTD_sp['Ticker Share Value'].cumsum()

# Cumulative sum of SP Share Value (latest FMV driven off of initial SP equiv purchase).
merged_portfolio_sp_latest_YTD_sp['Cum SP Returns'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Value'].cumsum()

# Cumulative CoC multiple return for stock investments
merged_portfolio_sp_latest_YTD_sp['Cum Ticker ROI Mult'] = merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] / merged_portfolio_sp_latest_YTD_sp['Cum Invst']

merged_portfolio_sp_latest_YTD_sp.head()


,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult
0,AAPL,2013-02-07,125,65.40,8175.0,2017-12-29,2018-03-09,175.405365,1.682039,1509.390015,...,13750.670624,6917.328747,164.258896,2673.610107,0.067859,0.04225,8175.0,21925.670624,15092.328747,2.682039
6,FB,2013-12-13,150,53.32,7998.0,2017-12-29,2018-03-09,185.229996,2.473931,1775.319946,...,19786.499359,4555.786405,176.460007,2673.610107,0.049700,0.04225,16173.0,49710.169983,27646.115152,3.073652
1,JNJ,2014-02-27,100,81.90,8190.0,2017-12-29,2018-03-09,127.506561,0.556857,1854.290039,...,4560.656128,4117.680233,132.300919,2673.610107,-0.036238,0.04225,24363.0,62460.826111,39953.795385,2.563758
2,MCD,2013-06-04,100,84.99,8499.0,2017-12-29,2018-03-09,150.689667,0.773028,1631.380005,...,6569.966675,6018.193382,163.912796,2673.610107,-0.080672,0.04225,32862.0,77529.792786,54470.988768,2.359254
3,MTCH,2015-12-14,600,13.63,8178.0,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,...,17399.618408,3092.646349,29.799627,2673.610107,0.430533,0.04225,41040.0,103107.411194,65741.635117,2.512364


## Assessing Where Positions are At versus Highest Close

In [45]:
# Referencing the adj_close dataframe from above

adj_close.head()

,Ticker,Date,Adj Close
0,AAPL,2013-01-02,68.850548
1,AAPL,2013-01-03,67.981491
2,AAPL,2013-01-04,66.087891
3,AAPL,2013-01-07,65.699158
4,AAPL,2013-01-08,65.875954


In [46]:
portfolio_df.head()

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
Ticker,,,,,
AAPL,2013-02-07,125,65.40,8175.00,2017-12-29
JNJ,2014-02-27,100,81.90,8190.00,2017-12-29
MCD,2013-06-04,100,84.99,8499.00,2017-12-29
MTCH,2015-12-14,600,13.63,8178.00,2017-12-29
NFLX,2016-01-14,75,108.71,8153.25,2017-12-29


In [47]:
# Need to factor in that some positions were purchased much more recently than others.
# Join adj_close dataframe with portfolio in order to have acquisition date.

portfolio_df.reset_index(inplace=True)

adj_close_acq_date = pd.merge(adj_close, portfolio_df, on='Ticker')

adj_close_acq_date.head()

,Ticker,Date,Adj Close,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
0,AAPL,2013-01-02,68.850548,2013-02-07,125,65.4,8175.0,2017-12-29
1,AAPL,2013-01-03,67.981491,2013-02-07,125,65.4,8175.0,2017-12-29
2,AAPL,2013-01-04,66.087891,2013-02-07,125,65.4,8175.0,2017-12-29
3,AAPL,2013-01-07,65.699158,2013-02-07,125,65.4,8175.0,2017-12-29
4,AAPL,2013-01-08,65.875954,2013-02-07,125,65.4,8175.0,2017-12-29


In [48]:
# delete_columns = ['Quantity', 'Unit Cost', 'Cost Basis', 'Start of Year']

del adj_close_acq_date['Quantity']
del adj_close_acq_date['Unit Cost']
del adj_close_acq_date['Cost Basis']
del adj_close_acq_date['Start of Year']

# Sort by these columns in this order in order to make it clearer where compare for each position should begin.
adj_close_acq_date.sort_values(by=['Ticker', 'Acquisition Date', 'Date'], ascending=[True, True, True], inplace=True)

In [49]:
# Anything less than 0 means that the stock close was prior to acquisition.
adj_close_acq_date['Date Delta'] = adj_close_acq_date['Date'] - adj_close_acq_date['Acquisition Date']

adj_close_acq_date['Date Delta'] = adj_close_acq_date[['Date Delta']].apply(pd.to_numeric)  

adj_close_acq_date.head()

,Ticker,Date,Adj Close,Acquisition Date,Date Delta
0,AAPL,2013-01-02,68.850548,2013-02-07,-3110400000000000
1,AAPL,2013-01-03,67.981491,2013-02-07,-3024000000000000
2,AAPL,2013-01-04,66.087891,2013-02-07,-2937600000000000
3,AAPL,2013-01-07,65.699158,2013-02-07,-2678400000000000
4,AAPL,2013-01-08,65.875954,2013-02-07,-2592000000000000


In [50]:
# Modified the dataframe being evaluated to look at highest close which occurred after Acquisition Date (aka, not prior to purchase).

adj_close_acq_date_modified = adj_close_acq_date[adj_close_acq_date['Date Delta']>=0]

adj_close_acq_date_modified.head()

,Ticker,Date,Adj Close,Acquisition Date,Date Delta
25,AAPL,2013-02-07,59.058853,2013-02-07,0
26,AAPL,2013-02-08,59.911518,2013-02-07,86400000000000
27,AAPL,2013-02-11,60.535877,2013-02-07,345600000000000
28,AAPL,2013-02-12,59.018475,2013-02-07,432000000000000
29,AAPL,2013-02-13,58.906216,2013-02-07,518400000000000


In [51]:
# This pivot table will index on the Ticker and Acquisition Date, and find the max adjusted close.

adj_close_pivot = adj_close_acq_date_modified.pivot_table(index=['Ticker', 'Acquisition Date'], values='Adj Close', aggfunc=np.max)

adj_close_pivot.reset_index(inplace=True)

adj_close_pivot

,Ticker,Acquisition Date,Adj Close
0,AAPL,2013-02-07,175.405365
1,FB,2013-12-13,193.089996
2,JNJ,2014-02-27,140.273819
3,MCD,2013-06-04,169.855286
4,MTCH,2015-12-14,42.629364
5,NFLX,2016-01-14,331.440002
6,TWTR,2015-01-05,52.869999
7,WMT,2013-08-14,104.885887


In [52]:
# Merge the adj close pivot table with the adj_close table in order to grab the date of the Adj Close High (good to know).

adj_close_pivot_merged = pd.merge(adj_close_pivot, adj_close
                                             , on=['Ticker', 'Adj Close'])

adj_close_pivot_merged.head()

,Ticker,Acquisition Date,Adj Close,Date
0,AAPL,2013-02-07,175.405365,2018-03-09
1,FB,2013-12-13,193.089996,2018-02-01
2,JNJ,2014-02-27,140.273819,2018-01-22
3,MCD,2013-06-04,169.855286,2018-01-26
4,MTCH,2015-12-14,42.629364,2018-03-09


In [53]:
# Merge the Adj Close pivot table with the master dataframe to have the closing high since you have owned the stock.

merged_portfolio_sp_latest_YTD_sp_closing_high = pd.merge(merged_portfolio_sp_latest_YTD_sp, adj_close_pivot_merged
                                             , on=['Ticker', 'Acquisition Date'])

# Renaming so that it's clear that the new columns are two year closing high and two year closing high date.
merged_portfolio_sp_latest_YTD_sp_closing_high.rename(columns={'Adj Close': 'Closing High Adj Close', 'Date': 'Closing High Adj Close Date'}, inplace=True)

merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp_closing_high['Closing High Adj Close'] - 1 

merged_portfolio_sp_latest_YTD_sp_closing_high

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult,Closing High Adj Close,Closing High Adj Close Date,Pct off High
0,AAPL,2013-02-07,125,65.40,8175.00,2017-12-29,2018-03-09,175.405365,1.682039,1509.390015,...,2673.610107,0.067859,0.04225,8175.00,21925.670624,15092.328747,2.682039,175.405365,2018-03-09,0.000000
1,FB,2013-12-13,150,53.32,7998.00,2017-12-29,2018-03-09,185.229996,2.473931,1775.319946,...,2673.610107,0.049700,0.04225,16173.00,49710.169983,27646.115152,3.073652,193.089996,2018-02-01,-0.040706
2,JNJ,2014-02-27,100,81.90,8190.00,2017-12-29,2018-03-09,127.506561,0.556857,1854.290039,...,2673.610107,-0.036238,0.04225,24363.00,62460.826111,39953.795385,2.563758,140.273819,2018-01-22,-0.091017
3,MCD,2013-06-04,100,84.99,8499.00,2017-12-29,2018-03-09,150.689667,0.773028,1631.380005,...,2673.610107,-0.080672,0.04225,32862.00,77529.792786,54470.988768,2.359254,169.855286,2018-01-26,-0.112835
4,MTCH,2015-12-14,600,13.63,8178.00,2017-12-29,2018-03-09,42.629364,2.127613,2021.939941,...,2673.610107,0.430533,0.04225,41040.00,103107.411194,65741.635117,2.512364,42.629364,2018-03-09,0.000000
5,NFLX,2016-01-14,75,108.71,8153.25,2017-12-29,2018-03-09,331.440002,2.048846,1921.839966,...,2673.610107,0.726610,0.04225,49193.25,127965.411377,77563.432360,2.601280,331.440002,2018-03-09,0.000000
6,TWTR,2015-01-05,225,36.38,8185.50,2017-12-29,2018-03-09,35.349998,-0.028312,2020.579956,...,2673.610107,0.472303,0.04225,57378.75,135919.161034,88852.007814,2.368807,52.869999,2015-04-07,-0.331379
7,WMT,2013-08-14,125,68.30,8537.50,2017-12-29,2018-03-09,85.449150,0.251086,1685.390015,...,2673.610107,-0.096213,0.04225,65916.25,146600.304794,102967.637877,2.224039,104.885887,2018-01-29,-0.185313


In [54]:
# Not needed for this blog post -- this is if you have multiple positions for the same ticker with different acquisition dates.
# merged_portfolio_sp_latest_YTD_sp_closing_high['Counts'] = merged_portfolio_sp_latest_YTD_sp_closing_high.index

# merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker #'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'].map(str) + ' ' + merged_portfolio_sp_latest_YTD_sp_closing_high['Counts'].map(str)

# merged_portfolio_sp_latest_YTD_sp_closing_high.head()

## YTD and Trailing Stop Charts

In [55]:
# Ploty is an outstanding resource for interactive charts.

trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp['Share YTD'][0:10],
    name = 'Ticker YTD')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp['SP 500 YTD'][0:10],
    name = 'SP500 YTD')
    
data = [trace1, trace2]

layout = go.Layout(title = 'YTD Return vs S&P 500 YTD'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [56]:
# Current Share Price versus Closing High Since Purchased

trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'][0:10],
    name = 'Pct off High')
    
data = [trace1]

layout = go.Layout(title = 'Adj Close % off of High'
    , barmode = 'group'
    , yaxis=dict(title='% Below Adj Close High', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

## Total Return Comparison Charts

In [57]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP Return'][0:10],
    name = 'SP500 Total Return')
    
data = [trace1, trace2]

layout = go.Layout(title = 'Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker', tickformat=".2%")
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

## Cumulative Returns Over Time

In [58]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Stock Gain / (Loss)'][0:10],
    name = 'Ticker Total Return ($)')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP 500 Gain / (Loss)'][0:10],
    name = 'SP 500 Total Return ($)')

trace3 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return %',
    yaxis='y2')

data = [trace1, trace2, trace3]

layout = go.Layout(title = 'Gain / (Loss) Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Gain / (Loss) ($)')
    , yaxis2=dict(title='Ticker Return', overlaying='y', side='right', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.75,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [59]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Invst'],
    # mode = 'lines+markers',
    name = 'Cum Invst')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum SP Returns'],
    # mode = 'lines+markers',
    name = 'Cum SP500 Returns')

trace3 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker Returns'],
    # mode = 'lines+markers',
    name = 'Cum Ticker Returns')

trace4 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker ROI Mult'],
    # mode = 'lines+markers',
    name = 'Cum ROI Mult'
    , yaxis='y2')


data = [trace1, trace2, trace3, trace4]

layout = go.Layout(title = 'Total Cumulative Investments Over Time'
    , barmode = 'group'
    , yaxis=dict(title='Returns')
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.4,y=1)
    , yaxis2=dict(title='Cum ROI Mult', overlaying='y', side='right')               
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)